# 預售社區基本資料
## 處理事項
- 由實價登錄網站抓取預售屋資料，每月更新
- 資料清洗

In [1]:
import os
import re
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

c:\pylabs\housing-market-insights


In [2]:
from utils.configs import COMMUNITY_BASE_URL, COMMUNITY_URLS_FRAGMENTS, COMMUNITY_COLUMN_NAME
from utils.helper_function import build_complete_urls, combined_df, csv_extractor  # 預售社區資料取得
from utils.helper_function import parse_admin_region, extract_mixed_alphanumeric_ids, extract_company_name, find_first_sale_time, convert_mixed_date_columns

## 從實價登錄網站取得預售屋社區資料
- function
    - build_complete_url(網址組合)、fetch_data(向實價網址請求資料)、combined_df(合併全台預售屋資料)

In [ ]:
# 組合預售屋社區網址
urls = build_complete_urls(COMMUNITY_BASE_URL, COMMUNITY_URLS_FRAGMENTS)
for idx, (city, link) in enumerate(urls.items(), start=1):
    print(f"{idx}. {city} → {link}")

print(f"\n總共有 {len(urls)} 筆縣市資料")

In [ ]:
# 依上述網址向實價登錄網站請求資料
raw_community_df = combined_df(urls, "20250801")

In [ ]:
# 儲存原始資料
output_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\ps_community\raw"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "raw_ps_community_20280801.csv"
out_path = os.path.join(output_dir,  csv_fn)
raw_community_df.to_csv(out_path, index=False, encoding='utf-8-sig')

## 載入原始資料並進行後續資料清洗
- function：csv_extractor(csv載入)

In [3]:
input_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\ps_community\raw"
csv_fn = "raw_ps_community_20280801.csv"
input_path = os.path.join(input_dir,  csv_fn)

In [4]:
# 載入原始預售屋社區資料
extracted =csv_extractor(input_path)
print(f" 逐筆交易資料載入成功: {extracted.shape}  記憶體使用: {extracted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
extracted.head()

total_rows: 8619


Extracting rows: 100%|█████████▉| 8596/8619 [00:00<00:00, 69959.73it/s]


Extracting finished
 逐筆交易資料載入成功: (8596, 27)  記憶體使用: 14.51 MB


,city_name,AA11,addr,apply,applydate,b,chkdate,city,e,f,...,ma,mark,name,pimg,pu,s,sn,subid,town,input_time
0,臺北市,第三種住宅區,大安區臥龍街268號,潤泰創新國際股份有限公司,1100702,辛亥段四小段482地號,1100819.0,A,"1100701~自完售止,1100701~售完為止",NaN,...,鋼筋混凝土造,潤泰創新國際股份有限公司,潤泰大安富陽,build_case.png,店舖、住宅,NaN,0,0,A02,20250801
1,臺北市,商業區,大同區延平北路一段25號旁,亞昕國際開發股份有限公司,1100702,"玉泉段一小段874地號,玉泉段一小段875地號,玉泉段一小段885地號,玉泉段一小段886地...",1100825.0,A,"1090601~1091231,事務所",1130709.0,...,鋼骨RC造,中國建築經理股份有限公司,亞昕華威,build_case.png,住商用,1100501~1111130,1,0,A09,20250801
2,臺北市,敦化南北路特定專用區(B區),松山區敦化北路56號,國美投資股份有限公司等5人,1100705,"敦化段二小段4地號,敦化段二小段4-1地號,敦化段二小段16地號,敦化段二小段32地號,敦化...",1120731.0,A,無,NaN,...,鋼骨造,台新建築經理股份有限公司,敦仰,build_case.png,一般零售業甲組(店舖);集合住宅,110年10月21日起至使照取得6個月止,2,2,A01,20250801
3,臺北市,第三種住宅區,士林區德行東路335號,朋記建設股份有限公司,1100705,"芝蘭段一小段320地號,芝蘭段一小段321地號,芝蘭段一小段325地號,芝蘭段一小段326地...",1101028.0,A,無,NaN,...,鋼筋混凝土造,台灣新光商業銀行股份有限公司,原禾賦,build_case.png,住商用,1100701~1101231,3,3,A15,20250801
4,臺北市,敦化南北路特定專用區(A)區,大安區建安里敦化南路一段161巷5弄6號2樓,富邦建設股份有限公司,1100705,"仁愛段六小段244地號,仁愛段六小段263-4地號,仁愛段六小段516地號,仁愛段六小段51...",1120721.0,A,至完銷,NaN,...,鋼骨造,富邦建設股份有限公司,富邦藝樹,build_case.png,住商用,1100301~1101031,4,2,A02,20250801


## 預售社區資料清洗
- 欄位名稱及型態轉換
- 新增行政區欄位: parse_admin_region
- 編號列表欄位拆解為編號清單及建設公司欄位: extract_mixed_alphanumeric_ids、extract_company_name
- 新增自售及代售的起始日期: find_first_sale_time
- 目期相關欄位轉換為datetime64: convert_mixed_date_columns
- 重複社區處理

In [5]:
proc_df = extracted.copy()

In [6]:
proc_df = proc_df.rename(columns=COMMUNITY_COLUMN_NAME, inplace=False)

In [7]:
# dataframe基本資料檢視
def info(df):
    """
    顯示 DataFrame 欄位資訊，包括資料型別、非空值數量、缺失數量與缺失率。
    參數：
    df (pd.DataFrame): 要檢視的資料表。
    """      
    print("-" * 50)
    summary = pd.DataFrame({
        '欄位名稱': df.columns,
        '資料型別': df.dtypes,
        '非空值數量': df.count(),
        '缺失值數量': df.isnull().sum(),
        '缺失率(%)': (df.isnull().sum() / len(df) * 100).round(2)
    }).reset_index(drop=True)

    return summary

In [ ]:
print("預售社區資料欄位資訊:")
display(info(proc_df))

In [8]:
# 新增行政區欄位
proc_df["行政區"] = proc_df["坐落街道"].apply(parse_admin_region)

# 新增欄位存放擷取後的編號清單
proc_df['備查編號清單'] = proc_df[ '編號列表'].apply(extract_mixed_alphanumeric_ids)

# 取出建商名稱
proc_df['建設公司'] = proc_df.apply(extract_company_name, axis=1)

# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
proc_df["自售起始日期"] = proc_df["自銷售期間"].apply(find_first_sale_time)
proc_df["代銷起始日期"] = proc_df["代銷售期間"].apply(find_first_sale_time)


# 日期欄位轉換
# 民國整數欄位 roc_integer_cols
# 民國斜線欄位（交易資料表）roc_slash_cols
# 西元欄位ad_cols
proc_df = convert_mixed_date_columns(
    proc_df,
    roc_cols=['完成建物第一次登記日期', '自售起始日期', '代銷起始日期', '備查完成日期', '建照核發日'],
    ad_cols=['匯入時間']
)

# proc_df.columns

In [ ]:
print("新增欄位後預售社區資料欄位資訊:")
display(info(proc_df))

In [9]:
# 儲存測式資料
output_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\ps_community\processed"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "ps_community_processed.csv"
out_path = os.path.join(output_dir,  csv_fn)
proc_df.to_csv(out_path, index=False, encoding='utf-8-sig')

In [ ]:
# 儲存測式資料
output_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\ps_community\temp"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "temp_ps_community_20280801.csv"
out_path = os.path.join(output_dir,  csv_fn)
proc_df.to_csv(out_path, index=False, encoding='utf-8-sig')

In [ ]:
def identify_duplicate_groups(df):
    """
    識別重複社區群組
    條件：行政區 + 建照執照 + 經度相同
    """
    duplicate_groups = {}
    group_id = 0
    
    # 建立分組條件
    df['group_key'] = df['行政區'].astype(str) + '|' + \
                     df['建照執照'].astype(str) + '|' + \
                     df['經度'].astype(str)
    
    # 找出重複群組
    group_counts = df['group_key'].value_counts()
    duplicate_keys = group_counts[group_counts > 1].index
    
    for key in duplicate_keys:
        indices = df[df['group_key'] == key]['編號'].tolist()
        if len(indices) > 1:
            duplicate_groups[group_id] = indices
            group_id += 1
    
    # 清理臨時欄位
    df.drop('group_key', axis=1, inplace=True)
    
    print(f"發現 {len(duplicate_groups)} 個重複群組")
    return duplicate_groups

In [ ]:
identify_duplicate_groups(proc_df)

In [ ]:
# 檢查是否有重複的社區組合
community_duplicates = proc_df[
    proc_df.duplicated(subset=['行政區','建照執照', '經度'], keep=False)
].sort_values(by=['行政區','建照執照', '經度'])

# 顯示結果
if not community_duplicates.empty:
    print("🔁 發現重複交易紀錄如下：")
    display(community_duplicates[['縣市','行政區', '建照執照', '經度', '社區名稱','戶數', '編號','備查編號清單', '建設公司', '自售起始日期', '代銷起始日期', '備查完成日期']])
else:
    print("✅ 沒有發現以『'行政區','建照執照'』為鍵的重複交易紀錄")

In [ ]:
def normalize_join(values, sep=', '):
    parts = []
    for v in values:
        if v is None:
            continue
        for p in re.split(r'[,\s]+', str(v)):
            if p:
                parts.append(p)
    out, seen = [], set()
    for p in parts:
        if p not in seen:
            out.append(p); seen.add(p)
    return sep.join(out)

src_id = 'G2H011112080002'  # 來源
dst_id = 'G2H011303070002'  # 目的

src = proc_df.loc[proc_df['編號'] == src_id, '備查編號清單']
dst = proc_df.loc[proc_df['編號'] == dst_id, '備查編號清單']

if not src.empty and not dst.empty:
    merged = normalize_join([dst.iloc[0], src.iloc[0]])
    proc_df.loc[proc_df['編號'] == dst_id, '備查編號清單'] = merged


In [ ]:
# proc_df.loc[proc_df['編號'] == src_id, '備查編號清單']
proc_df.loc[proc_df['編號'] == dst_id, '備查編號清單']


In [ ]:
proc_df[proc_df['編號'] == 'G2H011112080002']['備查編號清單']
proc_df[proc_df['編號'] == 'G2H011303070002']['備查編號清單']

In [ ]:
def identify_duplicate_groups(df):
    """
    識別重複社區群組
    條件：行政區 + 建照執照 + 經度相同
    """
    # 建立分組條件
    df['group_key'] = df['行政區'].astype(str) + '|' + \
                     df['建照執照'].astype(str) + '|' + \
                     df['經度'].astype(str)
    
    # 找出重複群組並直接建立字典
    duplicate_groups = (df[df.duplicated('group_key', keep=False)]
                       .groupby('group_key')['編號']
                       .apply(list)
                       .to_dict())
    
    # 重新編號
    duplicate_groups = {i: v for i, (k, v) in enumerate(duplicate_groups.items())}
    
    # 清理臨時欄位
    df.drop('group_key', axis=1, inplace=True)
    
    print(f"發現 {len(duplicate_groups)} 個重複群組")
    return duplicate_groups

In [ ]:
identify_duplicate_groups(proc_df)

In [ ]:
df_merged = (
    community_duplicates
    .groupby(['縣市','行政區','建照執照','經度'], as_index=False)
    .agg({
        '備查編號清單': lambda s: ', '.join(sorted(set(s))),
        '社區名稱':      lambda s: s.iloc[1] if len(s) > 1 else s.iloc[0],  # 取第2列,
        '戶數':          lambda s: s.iloc[1] if len(s) > 1 else s.iloc[0],  # 取第2列
        '編號':          lambda s:s.iloc[1] if len(s) > 1 else s.iloc[0],  # 取第2列,
        '建設公司':      lambda s: ', '.join(sorted(set(s))),
        '自售起始日期':  'first',
        '代銷起始日期':  'first',
        '備查完成日期':  'first',
    })
)
display(df_merged)

In [ ]:
display(info(proc_df))